# Testing your model works before submisson

In order to make sure that your model will work when tested, you should run this notebook. Take the following steps:
1. Have your 'model.dat' file placed in directory './saves/'. 
2. Paste the code of your CircleNet class in the second to last cell of this notebook. ** Make sure your CircleNet class code matches the code used to generate model.dat!**
3. Run this whole notebook
4. Verify that circles are painted in files located at "./test/draw" directory

In [1]:
import numpy as np
import PIL.Image as Image
from PIL import ImageDraw


import torch.nn as nn

import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import os
import glob
import datetime

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
class ShapesDataset(Dataset):
    
    def __init__(self, dataset_dir):
        """
        Initializing dataset by generating a dicitonary of labels, where an image file name is the key 
        and its labels are the contents of that entry in the dictionary. Images are not loaded. This way it
        is possible to iterate over arbitrarily large datasets (limited by labels dicitonary fitting 
        in memory, which is not a problem in practice)
        
        Args:
            dataset_dir : path to directory with images and labels. In this directory we expect to find
                          a directory called "images" containing the input images, and a file called 
                          "labels.txt" containing desired labels (coefficients)
        """
        
        self.dataset_dir = dataset_dir
        self.labels_dict = self.gen_labels_dict()
        self.images_keys = list(self.labels_dict)  # getting the keys of the dictionary as list
        self.images_keys.sort()                    # sorting so as to have in alphabetical order 

    def __len__(self):
        return len(self.labels_dict)

    def __getitem__(self, index):    
        """
        This funtion makes it possible to iterate over the ShapesDataset
        Args:
            index: running index of images
            
        Returns:
            sample: a dicitionary with three entries:
                    1. 'image'  contains the image
                    2. 'labels' contains labels (coeffs) corresponding to image
                    3. 'fname'  contains name of file (image_key) - may be useful for debugging
        """
        image_key = self.images_keys[index]     # recall - key is the file name of the corresponding image
        image = np.array(Image.open(image_key)) # image has shape: (128, 128, 3)
        image = image/255.0                     # simple normalization - just to maintain small numbers
        image = np.transpose(image, (2, 0, 1))  # network needs RGB channels to be first index
        labels = self.labels_dict[image_key]
        sample = {'image': image, 'labels': labels, 'fname':image_key}
        
        return sample
    
    
    def gen_labels_dict(self):
        """
        This fucntion generates a dictionary of labels
        
        Returns:
            labels_dict: the key is image file name and an array of labels is the corresponding contents 
        """
        
        labels_fname = self.dataset_dir + "/labels.txt"
        labels_dict = {}
        with open(labels_fname, "r") as inp:
            for line in inp:
                line = line.split('\n')[0]                                      # remove '\n' from end of line 
                line = line.split(',')
                key  = self.dataset_dir + '/images/' + line[0].strip() + ".png" # image file name is the key
                del line[0]
                
                list_from_line = [float(item) for item in line]
                labels_dict[key] = np.asarray(list_from_line, dtype=np.float32)
                        
        return labels_dict             


In [3]:
test_dir = "./test/"  


test_dataset = ShapesDataset(test_dir)

test_loader = DataLoader(test_dataset, 
                          batch_size=1,
                          shuffle=False)

In [4]:
def paint_loader_circles(model, loader, out_dir):
    """
    This fucntion receives a model, a loader and an output directory. For each image in the loader it paints
    a circle that the model identifies. The images are saved in the given out_dir diretory. 
    Args:
        model   - network for idneitfying circles
        loader  - input data to use 
        out_dir - ouptut directory name (e.g.: 'draws/'). If directory does not exist, it is created. If it exists,
                  its files are deleted.
    """

    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
                  # (dropout is set to zero)

    k = 0
    
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    files = glob.glob(out_dir + '*')
    for f in files:
        os.remove(f) 
  
        
    for data in loader:
        # get inputs
        inputs = (data['image']).to(device)
        labels = (data['labels']).to(device)  # not using 
        img_fnames = data['fname'] 
      
        
        # forward
        outputs = model(inputs.float())
        curr_batch_size = np.shape(outputs)[0]
        image_size = np.shape(inputs[0])  # image_size = [3, w, h]
        _, width, height = image_size
        assert (width == height)
        
        for i in range (curr_batch_size): 
            x0 = (outputs[i, 0].item()) * width
            y0 = (1-outputs[i, 1].item()) * height
            r  = outputs[i, 2].item() * width #assuming width=height here. Otherwise circle becomes ellipse.
   
            fname = img_fnames[i]
            k+=1
            print (str(k) + ".   " + fname)

            img = Image.open(fname)
            draw = ImageDraw.Draw(img, 'RGBA')
    
            draw.ellipse((x0 - r, y0 - r, x0 + r ,y0 + r), fill=(160, 64, 0, 90), outline=None)
    
            img.save(out_dir + fname.split('/')[-1])
    
        
    model.train()  #back to default
    return

# Your model definition is needed here!

In [5]:
import torch.nn.functional as F  # a lower level (compared to torch.nn) interface
import torchvision.transforms as transforms


class CircleNet(nn.Module):    # nn.Module is parent class  
    def __init__(self):
        super(CircleNet, self).__init__()  #calls init of parent class
                

        #----------------------------------------------
        # implementation needed here 
        #----------------------------------------------
      
        
        # The convolution layers were chosen to keep dimensions of input image: (I-F+2P)/S +1= (128-3+2)/1 + 1 = 128
        
        # Our images are RGB, so input channels = 3. Use 12 filters for first 2 convolution layers, then double
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=24, out_channels=32, kernel_size=3, stride=1, padding=1)

        
        
        #Pooling to reduce sizes, and dropout to prevent overfitting
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.relu = nn.ReLU()
        
        self.drop = nn.Dropout2d(p=0.2)
#         self.norm1 = nn.BatchNorm2d(12)
#         self.norm2 = nn.BatchNorm2d(24)
        
        # There are 2 pooling layers, each with kernel size of 2. Output size: 128/(2*2) = 32
        # Feature tensors are now 32 x 32; With 24 output channels, this gives 32x32x24
        
        # Have 3 output features, corresponding to x-pos, y-pos, radius. 
        self.fc = nn.Linear(in_features=32 * 32 * 32, out_features=3)
    

                
    def forward(self, x):
        """
        Feed forward through network
        Args:
            x - input to the network
            
        Returns "out", which is the network's output
        """
        
        #----------------------------------------------
        # implementation needed here 
        #----------------------------------------------
        
        #Convolution 1
        out = self.conv1(x)
        out = self.relu(out)
        out = self.pool(out)
        
#         out = self.norm1(out)

        #Convolution 2
        out = self.conv2(out)
        out = self.relu(out)
        out = self.pool(out)
        
#         out = self.norm1(out)
        
        #Convolution 3
        out = self.conv3(out)
        out = self.relu(out)
        out = self.drop(out)
#         out = self.norm2(out)
        
        #Convolution 4
        out = self.conv4(out)
        out = self.relu(out)
        out = F.dropout(out, training=self.training)
        
        
        out = out.view(-1, 32 * 32 * 32)
        out = self.fc(out)

        
        return out
             
             


In [6]:
model = torch.load("./saves/model.dat")  # make sure you place here the path to your model

paint_loader_circles(model, test_loader, './test/draw/')



1.   ./test//images/0000.png
2.   ./test//images/0001.png
3.   ./test//images/0002.png
4.   ./test//images/0003.png
5.   ./test//images/0004.png
6.   ./test//images/0005.png
7.   ./test//images/0006.png
8.   ./test//images/0007.png
9.   ./test//images/0008.png
10.   ./test//images/0009.png
11.   ./test//images/0010.png
12.   ./test//images/0011.png
13.   ./test//images/0012.png
14.   ./test//images/0013.png
15.   ./test//images/0014.png
16.   ./test//images/0015.png
17.   ./test//images/0016.png
18.   ./test//images/0017.png
19.   ./test//images/0018.png
20.   ./test//images/0019.png
21.   ./test//images/0020.png
22.   ./test//images/0021.png
23.   ./test//images/0022.png
24.   ./test//images/0023.png
25.   ./test//images/0024.png
26.   ./test//images/0025.png
27.   ./test//images/0026.png
28.   ./test//images/0027.png
29.   ./test//images/0028.png
30.   ./test//images/0029.png
31.   ./test//images/0030.png
32.   ./test//images/0031.png
33.   ./test//images/0032.png
34.   ./test//image